# USGS Earthquake Data Visualization using Bokeh (2018-08-30)

This Jupyter notebook pulls data from the USGS Earthquake Hazards portal and visualizes the dataset using the Bokeh library.

* USGS Earthquake data is retrieved as GeoJSON and then parsed into a dataframe for Bokeh

**USGS Data Resources**
* [USGS Earthquake Hazards Program](https://earthquake.usgs.gov/)
* [USGS Earthquake Hazards Program: GeoJSON Summary Format](https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php)
* [USGS Volcano Hazards Program](https://volcanoes.usgs.gov/observatories/hvo/hvo_earthquakes.html)

**Bokeh Visualization Library**
* [Bokeh](https://bokeh.pydata.org/en/latest/)
* [Bokeh Tutorial: Geographic Plots](https://hub.mybinder.org/user/bokeh-bokeh-notebooks-8dydb5oj/notebooks/tutorial/09%20-%20Geographic%20Plots.ipynb)

Note: When plotting geospatial data in Bokeh, WGS84 coordinates must be first converted to Web Mercator format in order to be visualized properly.

**Geospatial Resources**
* [World Geodetic System](https://en.wikipedia.org/wiki/World_Geodetic_System)
* [WGS84](https://wiki.gis.com/wiki/index.php/WGS84)
* [Web Mercator](https://en.wikipedia.org/wiki/Web_Mercator)

In [2]:
from datetime import datetime

import numpy as np
import pandas as pd
import requests

from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models import ColumnDataSource, HoverTool, BoxZoomTool, BoxSelectTool, CrosshairTool, SaveTool, PanTool, ResetTool


In [3]:
## USGS Earthquakes all earthquakes in the last week - Updated every 5 minutes
# url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson'
# url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson'
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson'

r = requests.get(url)

USGS_data = r.json()

In [4]:
temp = list()
k = 6378137

for quake in USGS_data['features']:

    if quake['properties']['mag'] != None:
        lon,lat,dep = quake['geometry']['coordinates']
        
        temp.append({'USGS_Id': quake['properties']['ids'][1:-1].upper(),
                     ## WGS84 Decimal Degrees
                     'Longitude': lon,
                     'Latitude': lat,
                     
                     ## Converts WGS84 Decimal Degrees to Web Mercator format
                     'x': lon * (k * np.pi/180.0),
                     'y': np.log(np.tan((90 + lat) * np.pi/360.0)) * k,
                     
                     'Depth': dep,
                     'Magnitude': quake['properties']['mag'], 
                     'LatLon': str(round(lat,2)) + '/' + str(round(lon,2)),
                     'Magnitude_Scaled': 5 + 6**(quake['properties']['mag']*0.4),
                     'Description': quake['properties']['title'],
                     'DTOI': datetime.fromtimestamp(quake['properties']['time']/1000).strftime('%d %b %y %H:%M:%SZ')})
        
df = pd.DataFrame(temp)

print(len(df))

259


In [5]:
output_file('USGS Earthquakes.html', title = 'USGS Earthquake Hazards Page')

source = ColumnDataSource(df)

# ## Configure the HoverTool popup dialog
hover = HoverTool(tooltips=[('USGS Id', '@USGS_Id'),
                            ('Date/Time', '@DTOI'),
                            ('Latitude/Longitude', '@LatLon'),
                            ('Magnitude', '@Magnitude'),
                            ('Description', '@Description')])

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(plot_width=1200, plot_height=800, 
           tools = [hover,BoxZoomTool(),BoxSelectTool(),CrosshairTool(),SaveTool(),PanTool(),ResetTool()], toolbar_location="above",
           title = 'USGS Reported Earthquakes',
           x_range=(-17900000, 17900000), y_range=(-5500000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")

# url = 'http://a.basemaps.cartocdn.com/rastertiles/voyager/{Z}/{X}/{Y}.png'
# attribution = "Tiles by Carto, under CC BY 3.0. Data by OSM, under ODbL"
# p.add_tile(WMTSTileSource(url=url, attribution=attribution))
p.add_tile(CARTODBPOSITRON)

# p.circle(x = df['x'], y = df['y'], size = df['Magnitude_Scaled'], source = df,
#          fill_color = 'red', fill_alpha = 0.2, line_color = None)

p.circle(x = 'x', y = 'y', size = 'Magnitude_Scaled', source = df,
         fill_color = 'red', fill_alpha = 0.2, line_color = None)

# p.circle('x', 'y', radius= 'Magnitude_Scaled', source = source,
#           fill_color='red', fill_alpha=0.2, line_color=None)

output_file('USGS Earthquakes.html', title = 'USGS Earthquake Hazards Page')
output_notebook()
show(p)  # open a browser

Loading BokehJS ...